## 建立詞頻矩陣

In [1]:
s = ['柯文哲為了大巨蛋一事找趙藤雄算帳',
     '柯P因為遠雄遲遲無法處理大巨蛋一事，因此在法庭上與趙藤雄對簿公堂',
     '大巨蛋今天舉辦世運會']

In [10]:
import jieba

In [17]:
jieba.load_userdict('userdict.txt')

In [18]:
corpus = []
for a in s:
    corpus.append(' '.join(jieba.cut(a)))

In [19]:
corpus

['柯文哲 為 了 大巨蛋 一事 找 趙藤雄 算帳',
 '柯P 因 為 遠雄 遲遲 無法 處理 大巨蛋 一事 ， 因此 在 法庭 上 與 趙藤雄 對簿 公堂',
 '大巨蛋 今天 舉辦 世運會']

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [27]:
print(vectorizer.get_feature_names())

['一事', '世運會', '今天', '公堂', '因此', '大巨蛋', '對簿', '柯p', '柯文哲', '法庭', '無法', '算帳', '舉辦', '處理', '趙藤雄', '遠雄', '遲遲']


In [24]:
X.toarray()

array([[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
       [1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1],
       [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]], dtype=int64)

In [25]:
X.shape

(3, 17)

In [28]:
from sklearn.metrics import euclidean_distances
ed = euclidean_distances(X)
ed

array([[0.        , 3.31662479, 2.64575131],
       [3.31662479, 0.        , 3.74165739],
       [2.64575131, 3.74165739, 0.        ]])

In [29]:
synonym_dic = {}
for s in open('synonym.txt', encoding='utf-8'):
    synonym = s.strip().split('/')
    for w in synonym[1:]:
        synonym_dic[w.lower()] = synonym[0]
synonym_dic

{'柯p': '柯文哲', 'kp': '柯文哲'}

In [30]:
class SynonymCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(SynonymCountVectorizer, self).build_analyzer()
        return lambda doc: (synonym_dic.get(w, w) for w in analyzer(doc))

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = SynonymCountVectorizer()
X = vectorizer.fit_transform(corpus)

In [32]:
print(vectorizer.get_feature_names())

['一事', '世運會', '今天', '公堂', '因此', '大巨蛋', '對簿', '柯文哲', '法庭', '無法', '算帳', '舉辦', '處理', '趙藤雄', '遠雄', '遲遲']


In [33]:
X.toarray()

array([[1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
       [1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1],
       [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]], dtype=int64)

In [34]:
ed = euclidean_distances(X)
ed

array([[0.        , 3.        , 2.64575131],
       [3.        , 0.        , 3.74165739],
       [2.64575131, 3.74165739, 0.        ]])

In [35]:
from sklearn.metrics.pairwise import cosine_distances
cosine_distances(X)

array([[0.        , 0.48360222, 0.7763932 ],
       [0.48360222, 0.        , 0.85566243],
       [0.7763932 , 0.85566243, 0.        ]])

In [36]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(X)

array([[1.        , 0.51639778, 0.2236068 ],
       [0.51639778, 1.        , 0.14433757],
       [0.2236068 , 0.14433757, 1.        ]])

## 新聞推薦

In [37]:
import pandas
df = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/cdc_course/master/data/20171003_applenews.xlsx')

In [39]:
corpus = []
titles = []
for rec in df.iterrows():
    corpus.append(' '.join(jieba.cut(rec[1].content)))
    titles.append(rec[1].title)

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [41]:
X.shape

(1020, 38072)

In [42]:
from sklearn.metrics.pairwise import cosine_distances
cs = cosine_distances(X)

In [43]:
cs.shape

(1020, 1020)

In [46]:
titles[2]

'北市中秋烤肉何處去\u3000這些地點讓你挑'

In [49]:
cs[2].argsort()

array([  2, 305, 178, ..., 290, 914, 328], dtype=int64)

In [52]:
for i in cs[2].argsort()[1:10]:
    print(titles[i], cs[2][i])

連假將至   3C賣場這支銷量暴增5倍 0.6986728861243147
【GAME啥】紅心辣椒旗下熱門遊戲推出「中秋國慶特別活動」 0.7355080245888544
【GAME啥】紅心辣椒旗下熱門遊戲推出「中秋國慶特別活動」 0.7355080245888544
便民措施　中橫便道明天新增2開放時段 0.7486128708111639
​中秋節烤肉「劫」 0.7518541665072674
​中秋節烤肉「劫」 0.7518541665072674
中秋賞月有利器　中原大學提供大砲級望遠鏡 0.7722783654170231
​【果動中秋】不只烤肉　烤肉架烤甜點才過癮 0.819045226447051
冒叛國危險！ 吳鳳來台灣破戒吃豬肉 0.835154470310036


In [53]:
def getSimiliarArticle(idx):
    print('查詢文章:', titles[idx])
    for i in cs[idx].argsort()[1:10]:
        print('相似文章:', titles[i], cs[idx][i])

In [57]:
getSimiliarArticle(6)

查詢文章: 阿羅哈6死車禍前　關鍵30秒影片曝光
相似文章: 阿羅哈6死車禍肇因出爐　駕駛分神找東西釀禍 0.5319427725863303
相似文章: 載陸客遊覽車誤闖機車道　撞水泥柱卡涵洞 0.8090530009660291
相似文章: 【慎入片】女駕駛出車禍　回神驚見友人嘴以上全部噴飛 0.8683627928825064
相似文章: 【驚魂片】轎車失控撞燈稈　駕駛、乘客受傷送醫 0.8757213945226147
相似文章: 鑽縫失敗碰碰碰　三寶騎士連撞3輛汽車 0.89300488698925
相似文章: 公車司機「霸道開法較安全」　怒！一堆人開嗆了 0.8973433635190913
相似文章: 驚！眼見愛車被拖吊　女抓車尾遭沿路拖行 0.9009169203893385
相似文章: 驚！深夜騎車狂飆　3年輕人撞車倒臥血泊 0.9049061719468796
相似文章: ​國道一號3車追撞　3人受傷送醫 0.9064735399596446


## 找出新聞議題

In [59]:
cs.shape

(1020, 1020)

In [63]:
from sklearn.cluster import KMeans
clf = KMeans(n_clusters = 10,  init='k-means++',)
c = clf.fit_predict(cs)

In [64]:
import numpy as np
np_titles = np.array(titles)

In [69]:
for ele in np_titles[c ==2 ]:
    print(ele)

賭城駭人槍擊　可能是美首宗用自動槍的大規模槍擊案
【賭城大屠殺】古巨基親述亡命6小時　藏匿其他旅客房
美國賭城大屠殺　外交部：已向美方表達慰問與關懷
【賭城大屠殺】生死交關5分鐘　靠「金手指」救命
R.I.P. 搖滾歌手湯姆佩蒂病逝　享壽66歲
中國大老闆遇賭城大屠殺　「練跑步很有用」
「以為是鞭炮」外媒拍下槍擊現場令人鼻酸　
​美國槍枝不管制 讓國民死在自己土地上 
【不斷更新】賭城59死大屠殺　槍手擁42把槍如軍火庫
臉書嘲諷槍擊案死者不值得同情　CBS高層遭炒
IS宣稱犯下賭城血案　專家指疑點重重
賭城大屠殺59死　王力宏籲禁槍「要多少無辜丟命」
賭城槍手反川普？　右翼網站操弄假新聞
美賭城槍擊案死傷慘重　蔡英文：譴責暴力
【懺悔片】支持擁槍者親歷屠殺才後悔　「無法形容過去錯誤」
​孔令信專欄：賭城屠殺 川普被自己人打臉
【賭城直擊】他出差遇到槍擊案　出事酒店一度封鎖
賭城槍擊案　我舊金山辦事處協尋失聯台人
【有片】因應美國賭城槍擊案　航警局加強桃機維安
【有片】因應美國賭城槍擊案　航警局加強桃機維安
【蘋果直擊】賭城生還者談救人：襲擊中互助令人動容
黃金周四川團賭城驚魂　「美劇槍擊情景重現」
賭城槍擊案後　美國再掀槍枝管制論戰
屠殺掃射速度驚人　槍手可能加了特殊裝置
賭城槍擊美史上最慘　護理師、軍警都遇害
【集氣片】明星齊悼賭城槍擊案　民眾大排長龍踴躍捐血
【賭城大屠殺】兇嫌弟：我哥超有錢　賭一手就賺750萬
白宮發言人談賭城哽咽　稱槍枝管制「言之過早」
【賭城大屠殺】夫捨身擋子彈　救妻一命
退休會計師賭城屠殺59死　父親名列十大通緝犯
【譴責片】沉重譴責賭城槍擊　川普：全然邪惡之舉
【賭城槍擊】如何幫受害者？捐血、捐物資資訊大整理
警尋獲賭城槍手亞裔女伴　事發時不在美國
避免淪為操弄對立工具　臉書將增聘1000名審查人力
【不斷更新】男賭城濫射　激增至58死逾500傷
【獨家】賭城大道民眾逃竄　台遊客驚呼：好像喪屍片
【獨家】賭城大道民眾逃竄　台遊客驚呼：好像喪屍片
沉重譴責賭城槍擊　川普：全然邪惡之舉
【快逃】「槍聲響不停」　賭城民眾驚險逃生影片曝光
賭城孤狼槍手飲彈自盡　飯店房裡搜到10把槍
港星古巨基賭城撞上槍擊案　報平安困守酒店
中共19大前夕 日美在台附近海域演習
音樂會最後一場浴血　歌手未放下吉他忙逃生
【獨家】名DJ瑪麗賭城槍擊現場驚魂！「便

In [73]:
cs.shape

(1020, 1020)

In [72]:
m = (cs < 0.5).astype(int)
m

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [74]:
m.sum()

1336

In [76]:
import networkx as nx
G = nx.from_numpy_matrix(m)

In [79]:
! pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org python-louvain

  Running setup.py bdist_wheel for python-louvain: started
  Running setup.py bdist_wheel for python-louvain: finished with status 'done'
  Stored in directory: D:\Users\nc20\AppData\Local\pip\Cache\wheels\f9\74\a9\14f051b00dddd46d71529db15507796da13a43ee2c0bc39def
Successfully built python-louvain


distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [84]:
import community
p = community.best_partition(G)

In [86]:
from collections import Counter
c = Counter(p.values())
c.most_common(10)

[(166, 14),
 (201, 7),
 (94, 5),
 (69, 4),
 (276, 4),
 (0, 3),
 (23, 3),
 (90, 3),
 (102, 3),
 (213, 3)]

In [95]:
np_cluster = np.array(list(p.values()))

In [101]:
for ele in np_titles[np_cluster == 69]:
    print(ele)

狗狗溜滑梯 滑稽動作有夠萌
貓咪緊追主人 原來爲了這目的
黑狗硬搶筆求陪玩 笑翻網友
【酥服片】主人貼心按摩 狗狗眯眼超享受


In [102]:
len(c)

910